In [11]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import math

In [19]:
def recommender(target, positive_list = None, negative_list = None, topn = 5):
    model = Word2Vec.load("item2Vec_20200422")
    recommend_anime_ls = []
    for animeId, prob in model.wv.most_similar_cosmul(positive = positive_list, negative = negative_list, topn = 9775):
        if int(animeId) in target:
            recommend_anime_ls.append(animeId)

    return recommend_anime_ls

In [16]:
anime = pd.read_csv("~/Data/anime.csv")
ls = recommender(positive_list = ["19859"], topn = 5)
print(anime[anime["anime_id"].isin(ls)])

       anime_id                                               name  \
0         32281                                     Kimi no Na wa.   
1          5114                   Fullmetal Alchemist: Brotherhood   
2         28977                                           Gintama°   
3          9253                                        Steins;Gate   
4          9969                                      Gintama&#039;   
...         ...                                                ...   
12287      9352                             Tenshi no Habataki Jun   
12288      5541                                   The Satisfaction   
12289      9316       Toushindai My Lover: Minami tai Mecha-Minami   
12290      5543                                        Under World   
12292      6133  Violence Gekiga Shin David no Hoshi: Inma Dens...   

                                                   genre   type episodes  \
0                   Drama, Romance, School, Supernatural  Movie        1   
1      

In [8]:
test_df = pd.read_csv("~/Thesis/Data/test.csv")
train_df = pd.read_csv("~/Thesis/Data/train.csv")

train_user_set, test_user_set = set(train_df.anime), set(test_df.anime)
not_user = test_user_set - (train_user_set & test_user_set)

animes = set(train_df[train_df.rating > 5].anime)
print(19859 in animes)

True


In [28]:
def ndcg(k, ranklist, testlist):
    if not testlist: return 0
    idcg_k, dcg_k = 0, 0
    if len(testlist) < k:
        k = len(testlist)
    for i in range(k):
        idcg_k += 1 / math.log(i + 2, 2)
        
    s = set(testlist)
    hits = [idx for idx, val in enumerate(ranklist) if val in s]
    count = len(hits)
    
    for i in range(count):
        dcg_k += 1 / math.log(hits[i] + 2, 2)
        
    return float(dcg_k / idcg_k)

In [ ]:
users = list(set(test_df.user.tolist()))
final = []
for k in range(9, 10):
    precision, ndcg_k = [], []
    for idx, user in enumerate(users):
        if not idx % 500: print(idx)
        target_items = test_df[(test_df.user == user)].anime.tolist()
        for i in range(10, -1, -1):
            items = test_df[(test_df.rating == i) & (test_df.user == user)].anime.tolist()
            if items and items[0] not in not_user and items[0] in animes:
                item = items[0]
                break
            
        rec = list(map(int, recommender(target = set(target_items), positive_list = [str(item)], topn = 5)[1:6]))
#         print(rec)
        cur_items = test_df[(test_df.rating > k) & (test_df.user == user)].anime.tolist()
#         precision.append(len(set(rec) & set(cur_items)) / len(rec))
        ndcg_k.append(ndcg(10, rec, cur_items))
        
    print(np.mean(ndcg_k))
    final.append(np.mean(ndcg_k))

0
500


In [25]:
print(final)

[1.0, 0.9977877047436716, 0.9937885556264625, 0.9871516698574773, 0.9736651776217827, 0.9432461178472666, 0.8780684960646671, 0.7226972984471388, 0.47292065517974896, 0.21748564135290363]
